In [37]:
'''import all necessary libraries'''

import cx_Oracle #import the oracle database libraries
import pandas as pd #connect to the pandas libraries
import numpy as np #import numpy
import getpass #import the password libraries in order to keep database password concealed

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import sys
print (sys.version)
pd.options.display.float_format = '{:0.6f}'.format

3.6.5 |Anaconda custom (64-bit)| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]


In [38]:
'''connect to the oracle server used to pull down the 
currency and misuse of authorization information'''

username=input("please enter your username")
password=getpass.getpass("Enter your password: ")

con=cx_Oracle.connect(username,password,'ODSPEU1.ams') #enter database details
print (con.version) # prnt the conversion details

please enter your username djohnson
Enter your password:  ··········


12.1.0.2.0


In [39]:
'''client names'''
df_ic=pd.read_sql(
''' 
select CP.CONTRACT_ID,CP.PRODUCT_CODE,CP.TRANSACTION_CURRENCY,CP.ICPLUS_TYPE,CP.PAYMENT_PROCESSOR
from FDWO.CC_PROCESSING_DATA_PT cp
where CP.PAYMENT_PROCESSOR='BARCLAYS'
and CP.MATCH_DATE between to_date('1/1/2018','MM/DD/YYYY') and to_date('8/31/2018','MM/DD/YYYY')
and CP.TRANSACTION_TYPE_SPECIFIC='TRANSACTION'
group by CP.CONTRACT_ID,CP.PRODUCT_CODE,CP.TRANSACTION_CURRENCY,CP.ICPLUS_TYPE,cp.PAYMENT_PROCESSOR
''', con)

In [174]:
columns=['MONTH','GROUP_NUMBER','CONTRACT_ID','CLIENT_NAME','TRANSACTION_CURRENCY_CODE'
         ,'BILLING_CURRENCY_CODE','CREDIT___DEBIT','STATEMENT_NARRATIVE'
         ,'ISSUER_REGION_DESCRIPTION','CARD_SCHEME_DESCRIPTION','MERCHANT_COUNTRY'
         ,'VOLUME','FLOW_EUR','SF_EUR'
         ,'PER_ITEM_RATE','PER_ITEM_CHARGE','PRODUCT_CODE']
df1=pd.read_csv('d:\\djohnson\\Desktop\\barclays_theoretical_dataset.txt',encoding='ISO-8859-1',sep=';',skiprows=31,low_memory=False,names=columns)

In [175]:
df2=pd.read_csv('G:\\Pricing\\Pricing Intelligence\\Market Changes\\Pricing Updates & Bulletins\\Scheme Fees\\Partners\\Barclays\\2018\\barclays_2018_fee_schedule_for_python.csv')
iep=pd.read_csv('d:\\djohnson\\Desktop\\aib_theoreticals_desktop_ref\\aib_iep_theoreticals_V3.csv',encoding='ISO-8859-1')
df_mc=pd.read_csv('d:\\djohnson\\Desktop\\aib_theoreticals_desktop_ref\\multicurrency.csv',encoding='ISO-8859-1')
df_cntry=pd.read_csv('d:\\djohnson\\Desktop\\barclays_sf_analysis\\barclays_missing_countries.csv',encoding='ISO-8859-1')

In [176]:
df_cntry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 2 columns):
CONTRACT_ID    44 non-null int64
COUNTRY        44 non-null object
dtypes: int64(1), object(1)
memory usage: 784.0+ bytes


In [177]:
df_ic=df_ic.rename(columns={'TRANSACTION_CURRENCY':'TRANSACTION_CURRENCY_CODE'})
df_ic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4011 entries, 0 to 4010
Data columns (total 5 columns):
CONTRACT_ID                  4011 non-null int64
PRODUCT_CODE                 4011 non-null int64
TRANSACTION_CURRENCY_CODE    4011 non-null object
ICPLUS_TYPE                  4011 non-null object
PAYMENT_PROCESSOR            4011 non-null object
dtypes: int64(2), object(3)
memory usage: 156.8+ KB


In [178]:
iep.head()

,PERIOD,Concatenate,Country,Regionality,Product,Security Level,Fixed,Percent
0,prejuly18,VisaDebitAustriaDomestic,Austria,Domestic,VisaDebit,Nonsecure,0.008200,0.000200
1,prejuly18,VisaDebitBelgiumDomestic,Belgium,Domestic,VisaDebit,Nonsecure,0.008200,0.000200
2,prejuly18,VisaDebitBulgariaDomestic,Bulgaria,Domestic,VisaDebit,Nonsecure,0.008200,0.000200
3,prejuly18,VisaDebitCroatiaDomestic,Croatia,Domestic,VisaDebit,Nonsecure,0.008200,0.000200
4,prejuly18,VisaDebitCyprusDomestic,Cyprus,Domestic,VisaDebit,Nonsecure,0.008200,0.000200


In [179]:
df2.head()

,PERIOD,CHECK,GROUP_NUMBER,CREDIT___DEBIT,ISSUER_REGION_DESCRIPTION,SCHEDULE_REGION,CARD_SCHEME_DESCRIPTION,PERC_RATE,FIXED_RATE
0,prejuly18,Y,8765,CR,Intra-Region,Intra-Region,Visa,0.000240,0.016790
1,prejuly18,Y,4446,CR,Intra-Region,Intra-Region,Visa,0.000240,0.016790
2,prejuly18,Y,4446,CR,Domestic,Domestic,Visa,0.000240,0.008200
3,prejuly18,Y,2417,CR,Domestic,Domestic,MasterCard,0.000250,0.003850
4,prejuly18,Y,7101,CR,Intra-Region,Intra-Region,Maestro,0.000417,0.004810


In [180]:
df1_working=df1.copy()
df2_working=df2[['GROUP_NUMBER','CREDIT___DEBIT','ISSUER_REGION_DESCRIPTION','CARD_SCHEME_DESCRIPTION','PERC_RATE','FIXED_RATE']].copy()

In [181]:
df1_working.STATEMENT_NARRATIVE.unique()

array(['Credit ', 'Debit  ', 'Maestro', 'Vpay   '], dtype=object)

In [182]:
df1_working['CLIENT_NAME']=df1_working['CLIENT_NAME'].str.strip()
df1_working['TRANSACTION_CURRENCY_CODE']=df1_working['TRANSACTION_CURRENCY_CODE'].str.strip()
df1_working['BILLING_CURRENCY_CODE']=df1_working['BILLING_CURRENCY_CODE'].str.strip()
df1_working['CREDIT___DEBIT']=df1_working['CREDIT___DEBIT'].str.strip()
df1_working['STATEMENT_NARRATIVE']=df1_working['STATEMENT_NARRATIVE'].str.strip()
df1_working['ISSUER_REGION_DESCRIPTION']=df1_working['ISSUER_REGION_DESCRIPTION'].str.strip()
df1_working['CARD_SCHEME_DESCRIPTION']=df1_working['CARD_SCHEME_DESCRIPTION'].str.strip()
df1_working['MERCHANT_COUNTRY']=df1_working['MERCHANT_COUNTRY'].str.strip()
df1_working['CREDIT___DEBIT']=df1_working['CREDIT___DEBIT'].str.strip()
df1_working['FLOW_EUR']=df1_working['FLOW_EUR'].str.strip()

In [183]:
'''add in the missing countries merchant list'''
df1_working=pd.merge(df1_working,df_cntry,on='CONTRACT_ID',how='left').copy()

In [184]:
for x in range(0,len(df1_working['MONTH'])):
    if df1_working.loc[x,'MERCHANT_COUNTRY']=='':
        df1_working.loc[x,'MERCHANT_COUNTRY']=df1_working.loc[x,'COUNTRY']

In [185]:
'''Adjust the regionality to ensure that certain countries are being assessed the correct scheme fees'''

df1_working['MERCHANT_COUNTRY'][(df1_working['MERCHANT_COUNTRY']=='United Kingdom')]='UK'

df1_working['MERCHANT_COUNTRY'][(df1_working['MERCHANT_COUNTRY']=='Russian Federation')&(df1_working['CARD_SCHEME_DESCRIPTION']=='MasterCard')]='Russia'
df1_working['MERCHANT_COUNTRY'][(df1_working['MERCHANT_COUNTRY']=='Russian Federation')&(df1_working['CARD_SCHEME_DESCRIPTION']=='Maestro')]='Russia'

df1_working['MERCHANT_COUNTRY_UPDATED_FROM:']=''
df1_working['MERCHANT_COUNTRY_UPDATED_FROM:'][(df1_working['MERCHANT_COUNTRY']=='Gibraltar')]='Gibraltar'
df1_working['MERCHANT_COUNTRY_UPDATED_FROM:'][(df1_working['MERCHANT_COUNTRY']=='Andorra')]='Andorra'
df1_working['MERCHANT_COUNTRY_UPDATED_FROM:'][(df1_working['MERCHANT_COUNTRY']=='Panama')]='Panama'
df1_working['MERCHANT_COUNTRY_UPDATED_FROM:'][(df1_working['MERCHANT_COUNTRY']=='Monaco')]='Monaco'
df1_working['MERCHANT_COUNTRY_UPDATED_FROM:'][(df1_working['MERCHANT_COUNTRY']=='Australia')]='Australia'
df1_working['MERCHANT_COUNTRY_UPDATED_FROM:'][(df1_working['MERCHANT_COUNTRY']=='New Zealand')]='New Zealand'
df1_working['MERCHANT_COUNTRY_UPDATED_FROM:'][(df1_working['MERCHANT_COUNTRY']=='Singapore')]='Singapore'
df1_working['MERCHANT_COUNTRY_UPDATED_FROM:'][(df1_working['MERCHANT_COUNTRY']=='Thailand')]='Thailand'
df1_working['MERCHANT_COUNTRY_UPDATED_FROM:'][(df1_working['MERCHANT_COUNTRY']=='Malaysia')]='Malaysia'

df1_working['MERCHANT_COUNTRY'][(df1_working['MERCHANT_COUNTRY']=='Gibraltar')]='UK'
df1_working['MERCHANT_COUNTRY'][(df1_working['MERCHANT_COUNTRY']=='Andorra')]='France'
df1_working['MERCHANT_COUNTRY'][(df1_working['MERCHANT_COUNTRY']=='Panama')]='Ireland'
df1_working['MERCHANT_COUNTRY'][(df1_working['MERCHANT_COUNTRY']=='Monaco')]='France'
df1_working['MERCHANT_COUNTRY'][(df1_working['MERCHANT_COUNTRY']=='Australia')]='Ireland'
df1_working['MERCHANT_COUNTRY'][(df1_working['MERCHANT_COUNTRY']=='New Zealand')]='Ireland'
df1_working['MERCHANT_COUNTRY'][(df1_working['MERCHANT_COUNTRY']=='Singapore')]='Ireland'
df1_working['MERCHANT_COUNTRY'][(df1_working['MERCHANT_COUNTRY']=='Thailand')]='Ireland'
df1_working['MERCHANT_COUNTRY'][(df1_working['MERCHANT_COUNTRY']=='Malaysia')]='Ireland'

C:\Users\djohnson\new_python_version\Local\Continuum\anaconda2\envs\py3k\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\djohnson\new_python_version\Local\Continuum\anaconda2\envs\py3k\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\djohnson\new_python_version\Local\Continuum\anaconda2\envs\py3k\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http:/

In [186]:
df1_working=pd.merge(df1_working,df_mc,on=['MERCHANT_COUNTRY'],how='left').copy()

In [187]:
df1_working['GROUP_NUMBER']=df1_working['GROUP_NUMBER'].astype(float)
df2_working['GROUP_NUMBER']=df2_working['GROUP_NUMBER'].astype(float)

In [188]:
#df1_working.to_csv('d:\\djohnson\\desktop\\df1_working.csv')
df1_working=df1_working[(df1_working['FLOW_EUR']!='')].copy()
df1_working['FLOW_EUR']=df1_working['FLOW_EUR'].astype(float)
df1_working=df1_working.reset_index(drop=True).copy()

In [189]:
for x in range(0,len(df1_working)):
    if 'Credit' in df1_working.loc[x,'STATEMENT_NARRATIVE']:
        df1_working.loc[x,'CREDIT___DEBIT']='CR'
    if 'Debit' in df1_working.loc[x,'STATEMENT_NARRATIVE']:
        df1_working.loc[x,'CREDIT___DEBIT']='DR'
    if 'Vpay' in df1_working.loc[x,'STATEMENT_NARRATIVE']:
        df1_working.loc[x,'CREDIT___DEBIT']='CR'
    if 'Maestro' in df1_working.loc[x,'STATEMENT_NARRATIVE']:
        df1_working.loc[x,'CREDIT___DEBIT']='CR'

In [190]:
df1_working.head()

,MONTH,GROUP_NUMBER,CONTRACT_ID,CLIENT_NAME,TRANSACTION_CURRENCY_CODE,BILLING_CURRENCY_CODE,CREDIT___DEBIT,STATEMENT_NARRATIVE,ISSUER_REGION_DESCRIPTION,CARD_SCHEME_DESCRIPTION,MERCHANT_COUNTRY,VOLUME,FLOW_EUR,SF_EUR,PER_ITEM_RATE,PER_ITEM_CHARGE,PRODUCT_CODE,COUNTRY,MERCHANT_COUNTRY_UPDATED_FROM:,MULTI_CURRENCY_APPLICABLE
0,10,7101.000000,1239,Barcelo,EUR,EUR,CR,Credit,Domestic,MasterCard,Spain,915,190403.460000,34.980000,0.004000,0.017000,100450.000000,Spain,,Yes
1,10,7101.000000,1239,Barcelo,EUR,EUR,DR,Debit,Intra-Region,Visa,Spain,31,12916.490000,73.950000,0.052000,0.560000,100440.000000,Spain,,Yes
2,10,7101.000000,1239,Barcelo,EUR,EUR,CR,Credit,Intra-Region,MasterCard,Spain,28,12639.110000,7.890000,0.004000,0.062000,100450.000000,Spain,,Yes
3,10,7101.000000,1239,Barcelo,EUR,EUR,DR,Debit,Intra-Region,MasterCard,Spain,16,6211.620000,3.890000,0.004000,0.062000,100450.000000,Spain,,Yes
4,10,7101.000000,1239,Barcelo,USD,EUR,CR,Credit,Intra-Region,Visa,Spain,1,525.559000,0.150000,0.004000,0.024000,100443.000000,Spain,,Yes


In [191]:
df2_working.head()

,GROUP_NUMBER,CREDIT___DEBIT,ISSUER_REGION_DESCRIPTION,CARD_SCHEME_DESCRIPTION,PERC_RATE,FIXED_RATE
0,8765.000000,CR,Intra-Region,Visa,0.000240,0.016790
1,4446.000000,CR,Intra-Region,Visa,0.000240,0.016790
2,4446.000000,CR,Domestic,Visa,0.000240,0.008200
3,2417.000000,CR,Domestic,MasterCard,0.000250,0.003850
4,7101.000000,CR,Intra-Region,Maestro,0.000417,0.004810


In [192]:
master_data_set=pd.merge(df1_working,df2_working,how='left',on=['GROUP_NUMBER','CREDIT___DEBIT','ISSUER_REGION_DESCRIPTION','CARD_SCHEME_DESCRIPTION'])

In [193]:
master_data_set['MULTI_CURRENCY_FEE']=0

master_data_set['MULTI_CURRENCY_FEE'][(master_data_set['MULTI_CURRENCY_APPLICABLE']=='Yes')&(master_data_set['CARD_SCHEME_DESCRIPTION']=='MasterCard')&(master_data_set['ISSUER_REGION_DESCRIPTION']=='Domestic')&(master_data_set['TRANSACTION_CURRENCY_CODE']!=master_data_set['BILLING_CURRENCY_CODE'])]=.0003
master_data_set['MULTI_CURRENCY_FEE'][(master_data_set['MULTI_CURRENCY_APPLICABLE']=='Yes')&(master_data_set['CARD_SCHEME_DESCRIPTION']=='Maestro')&(master_data_set['ISSUER_REGION_DESCRIPTION']=='Domestic')&(master_data_set['TRANSACTION_CURRENCY_CODE']!=master_data_set['BILLING_CURRENCY_CODE'])]=.0003

master_data_set['MULTI_CURRENCY_FEE'][(master_data_set['MULTI_CURRENCY_APPLICABLE']=='Yes')&(master_data_set['CARD_SCHEME_DESCRIPTION']=='MasterCard')&(master_data_set['ISSUER_REGION_DESCRIPTION']=='Intra-Region')&(master_data_set['TRANSACTION_CURRENCY_CODE']!=master_data_set['BILLING_CURRENCY_CODE'])]=.0003
master_data_set['MULTI_CURRENCY_FEE'][(master_data_set['MULTI_CURRENCY_APPLICABLE']=='Yes')&(master_data_set['CARD_SCHEME_DESCRIPTION']=='Maestro')&(master_data_set['ISSUER_REGION_DESCRIPTION']=='Intra-Region')&(master_data_set['TRANSACTION_CURRENCY_CODE']!=master_data_set['BILLING_CURRENCY_CODE'])]=.0003

master_data_set['MULTI_CURRENCY_FEE'][(master_data_set['MULTI_CURRENCY_APPLICABLE']=='Yes')&(master_data_set['CARD_SCHEME_DESCRIPTION']=='MasterCard')&(master_data_set['ISSUER_REGION_DESCRIPTION']=='Inter-Region')&(master_data_set['TRANSACTION_CURRENCY_CODE']!=master_data_set['BILLING_CURRENCY_CODE'])]=.0013
master_data_set['MULTI_CURRENCY_FEE'][(master_data_set['MULTI_CURRENCY_APPLICABLE']=='Yes')&(master_data_set['CARD_SCHEME_DESCRIPTION']=='Maestro')&(master_data_set['ISSUER_REGION_DESCRIPTION']=='Inter-Region')&(master_data_set['TRANSACTION_CURRENCY_CODE']!=master_data_set['BILLING_CURRENCY_CODE'])]=.0013

C:\Users\djohnson\new_python_version\Local\Continuum\anaconda2\envs\py3k\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\djohnson\new_python_version\Local\Continuum\anaconda2\envs\py3k\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\djohnson\new_python_version\Local\Continuum\anaconda2\envs\py3k\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the ca

In [194]:
master_data_set['BARCLAYS_THEORETICAL_SF']=(master_data_set['FLOW_EUR']*master_data_set['PERC_RATE'])+(master_data_set['VOLUME']*master_data_set['FIXED_RATE'])

In [195]:
master_data_set=pd.merge(master_data_set,df_ic,how='left',on=['CONTRACT_ID','TRANSACTION_CURRENCY_CODE','PRODUCT_CODE'])

In [196]:
master_data_set['MULTI_CURRENCY_APPLICABLE'][(master_data_set['MULTI_CURRENCY_FEE']==0)]='No'

C:\Users\djohnson\new_python_version\Local\Continuum\anaconda2\envs\py3k\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [197]:
master_data_set['Product']=''
for x in range(0,len(master_data_set['Product'])):
    if master_data_set.loc[x,'CREDIT___DEBIT']=='DR':
        master_data_set.loc[x,'Product']=master_data_set.loc[x,'CARD_SCHEME_DESCRIPTION']+'Debit'
    else:
        master_data_set.loc[x,'Product']=master_data_set.loc[x,'CARD_SCHEME_DESCRIPTION']

In [198]:
master_data_set['Concatenate']=master_data_set['Product']+master_data_set['MERCHANT_COUNTRY']+master_data_set['ISSUER_REGION_DESCRIPTION']

In [199]:
'''build period of time which will be used to determine which fees will be applied'''
master_data_set['PERIOD']=''
for x in range(0,len(master_data_set['MONTH'])):
    if master_data_set.loc[x,'MONTH']<7:
        master_data_set.loc[x,'PERIOD']='prejuly18'
    else:
        master_data_set.loc[x,'PERIOD']='postjuly18'

In [200]:
master_data_set['Security Level']='Nonsecure'

In [201]:
master_data_set=pd.merge(master_data_set,iep,on=['Security Level','Concatenate','PERIOD'],how='left').copy()

In [202]:
master_data_set['IEP_THEORETICAL_SF']=(master_data_set['FLOW_EUR']*master_data_set['Percent'])+(master_data_set['VOLUME']*master_data_set['Fixed'])+(master_data_set['MULTI_CURRENCY_FEE']*master_data_set['FLOW_EUR'])

In [203]:
master_data_set=master_data_set.drop_duplicates().copy()

In [204]:
master_data_set=master_data_set.rename(columns={
    "CREDIT___DEBIT":"Credit - Debit",
    "Fixed":"IeP Fixed Rate",
    "Percent":"IeP Percent Rate",
    "IEP_THEORETICAL_SF":"IeP Theoretical SF",
    "SF_EUR":"Barclays Actual Scheme Fees",
    "PER_ITEM_RATE":"Barclays Actual Fixed Rate",
    "PER_ITEM_CHARGE":"Barclays Actual Percent Rate",
    "BARCLAYS_THEORETICAL_SF":"Barclays theoretical Scheme Fees",
    "PERC_RATE":"Barclays theoretical Fixed Rate",
    "FIXED_RATE":"Barclays theoretical Percent Rate",
    "CARD_SCHEME_DESCRIPTION":"Card Scheme",
    "MERCHANT_COUNTRY":"Merchant Country"})

In [205]:
master_data_set=master_data_set.drop(['Product_y','COUNTRY'
                                      ,'PAYMENT_PROCESSOR'
                                      ,'Product_x','Concatenate'
                                      ,'PERIOD','ISSUER_REGION_DESCRIPTION'],axis=1).copy()

In [206]:
master_data_set=master_data_set.reset_index(drop=True).copy()

In [207]:
for x in range(0,len(master_data_set['Barclays Actual Percent Rate'])):
    master_data_set.loc[x,'Barclays Actual Percent Rate']=master_data_set.loc[x,'Barclays Actual Percent Rate']/100

In [208]:
master_data_set=master_data_set[(master_data_set['Merchant Country']!='United States')].copy()

In [209]:
master_data_set=master_data_set.reset_index(drop=True).copy()

In [210]:
for x in range(0,len(master_data_set['ICPLUS_TYPE'])):
    if master_data_set.loc[x,'ICPLUS_TYPE']=='-':
        master_data_set.loc[x,'ICPLUS_TYPE']='Blended'

In [212]:
master_data_set.to_csv('d:\\djohnson\\Desktop\\barclays_sf_analysis\\barclays_sf_analysis.csv')